In [17]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
from sklearn.impute import KNNImputer

##### Versiseti Kaynak: https://www.kaggle.com/datasets/alexandrepetit881234/fake-bills

- is_genuine: Bir banknotun gerçek (genuine) mi yoksa sahte (fake) mi olduğunu belirten bir özellik

- diagonal: Banknotun köşegen uzunluğunu mm cinsinden ifade eden bir özellik

- height_left: Banknotun sol yüksekliğini mm cinsinden belirten bir özellik

- height_right: Banknotun sağ yüksekliğini mm cinsinde nbelirten bir özellik

- margin_low: Banknotun alt kenarındaki boşluğu mm cinsinden ifade eden bir özellik 

- margin_up: Banknotun üst kenarındaki boşluğu mm cinsinden ifade eden bir özellik olabilir.

- length: Banknotun tüm uzunluğunu mm cinsinden ifade eden bir özellik

In [18]:
base_url = "C:/Users/M E L S A/Desktop/M E L S A/data_science/datasciencetemp/data/bills/"
bills_dataset_url = base_url + "fake_bills.csv"

In [19]:
df = pd.read_csv(bills_dataset_url) #url yi oku ve DataFrame objesine ata
df.head()

,is_genuine,diagonal,height_left,height_right,margin_low,margin_up,length
0,True,171.81,104.86,104.95,4.52,2.89,112.83
1,True,171.46,103.36,103.66,3.77,2.99,113.09
2,True,172.69,104.48,103.50,4.40,2.94,113.16
3,True,171.36,103.91,103.94,3.62,3.01,113.51
4,True,171.73,104.28,103.46,4.04,3.48,112.54


In [21]:
df.isnull().sum() # eksik verinin kolon bazında saydırılması

is_genuine       0
diagonal         0
height_left      0
height_right     0
margin_low      37
margin_up        0
length           0
dtype: int64

#### Eksik Verilerin KNNImputer ile Doldurulması
##### KNNImputer; eksik değerlere sahip olan her bir veri noktasını, bu noktanın K en yakın komşusunun değerleri ile doldurur
##### 1 - Eksik değeri doldurulacak noktanın konumu belirlenir:
- Eksik değeri olan veri noktası belirlenir.
##### 2 - Eksik değeri olan noktanın K en yakın komşusu bulunur:
- Diğer tüm noktalara olan uzaklıklar hesaplanır ve en yakın K nokta belirlenir. Bu uzaklık genellikle Euclidean mesafesi kullanılarak hesaplanır.
##### 3 - Eksik değeri, K en yakın komşunun değerleri kullanılarak doldurulur:
- Eksik değeri olan noktanın eksik değeri, K en yakın komşularının ilgili özelliklerinin ortalaması, medyanı veya ağırlıklı ortalaması ile doldurulabilir.
##### Bu yöntem, bir tür "ortak anlayış" yaklaşımını temsil eder: eksik değeri, benzer özelliklere sahip diğer noktalardan alınan bilgilerle tamamlamaya çalışır. Bu şekilde, eksik verileri doldururken çevresel bağlamı ve veri setinin genel özelliklerini dikkate alabilir.

In [26]:
# Eksik değerleri içeren sütunun adı
missing_column = 'margin_low'

# KNNImputer oluşturulur
imputer = KNNImputer(n_neighbors=5)  # n_neighbors: K değeri, yani komşuluk sayısı

# Eksik değerler doldurulur
df[[missing_column]] = imputer.fit_transform(df[[missing_column]])

# Eksik değerleri doldurulmuş veri görüntülenir
print(df)

      is_genuine  diagonal  height_left  height_right  margin_low  margin_up  \
0           True    171.81       104.86        104.95        4.52       2.89   
1           True    171.46       103.36        103.66        3.77       2.99   
2           True    172.69       104.48        103.50        4.40       2.94   
3           True    171.36       103.91        103.94        3.62       3.01   
4           True    171.73       104.28        103.46        4.04       3.48   
...          ...       ...          ...           ...         ...        ...   
1495       False    171.75       104.38        104.17        4.42       3.09   
1496       False    172.19       104.63        104.44        5.27       3.37   
1497       False    171.80       104.01        104.12        5.51       3.36   
1498       False    172.06       104.28        104.06        5.17       3.46   
1499       False    171.47       104.15        103.82        4.63       3.37   

      length  
0     112.83  
1     113

In [27]:
df.isnull().sum()

is_genuine      0
diagonal        0
height_left     0
height_right    0
margin_low      0
margin_up       0
length          0
dtype: int64

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [40]:
# Bağımlı değişken (etiket): is_genuine (1: Gerçek, 0: Sahte)
y = df['is_genuine']

# Bağımsız değişkenler
X = df.drop('is_genuine', axis=1)

In [41]:
# Veriyi eğitim ve test setlerine bölelim
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)

In [42]:
# Ölçeklendirmeyi uygula (özelliklerin ölçeklendirilmesi genellikle KNN'de önerilir)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [43]:
# KNN sınıflandırıcı modelini oluştur
knn_model = KNeighborsClassifier(n_neighbors=5)  # Örnek olarak, k=5 kullanıldı. Bu değeri değiştirebilirsiniz.

In [44]:
# Modeli eğit
knn_model.fit(X_train_scaled, y_train)

KNeighborsClassifier()

In [45]:
# Test setini kullanarak tahmin yap
y_pred = knn_model.predict(X_test_scaled)

In [46]:
# Model performansını değerlendir
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

In [47]:
# Sonuçları görüntüle
print(f'Accuracy: {accuracy:.2f}\n')
print('Confusion Matrix:')
'''
[[TrueNegative, FalsePositive],
 [FalseNesgative, TruePositive]]
'''
print(conf_matrix)
print('\nClassification Report:')
print(class_report)

Accuracy: 1.00

Confusion Matrix:
[[106   1]
 [  0 193]]

Classification Report:
              precision    recall  f1-score   support

       False       1.00      0.99      1.00       107
        True       0.99      1.00      1.00       193

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300



##### K-En Yakın Komşu (KNN) algoritması için uygulanabilir değerlendirme ölçütleri arasında Jaccard İndeksi, Log Kaybı (Log Loss) ve F1 Skoru bulunmaktadır. Bu metrikler, sınıflandırma algoritmalarının performansını değerlendirmek için kullanılır.

##### Jaccard İndeksi:
##### Tanım: Jaccard İndeksi, bir sınıflandırma modelinin tahmin ettiği ve gerçekte olan sınıflar arasındaki benzerliği ölçer. Şu formülle ifade edilir: 
- J(A,B)= ∣A∪B∣/∣A∩B∣
##### Açıklama: İndeks, gerçekte ve tahmin edilen sınıfların kesişimini gerçekte ve tahmin edilen sınıfların birleşimine böler. Değer 1'e yaklaştıkça, tahminler gerçek değerlere daha yakındır.

##### Log Kaybı (Log Loss):
##### Tanım: Log Kaybı, sınıflandırma modelinin tahmin ettiği olasılıklar ile gerçekte olan sınıf etiketleri arasındaki farkı ölçer.
##### Açıklama: Düşük log kaybı değerleri, daha iyi bir model performansını gösterir.

##### F1 Skoru:
##### Tanım: F1 Skoru, precision ve recall metriklerinin harmonik ortalamasıdır ve şu formülle ifade edilir: 
- F1=2x (PrecisionxRecall/Precision+Recall)
##### Açıklama: F1 skoru, hassasiyet (precision) ve duyarlılık (recall) metriklerinin dengesini ölçer. Değer 1'e yaklaştıkça, modelin sınıfları ne kadar iyi ayırt ettiğini gösterir.

##### Bu metrikler, sınıflandırma modellerinin performansını değerlendirmek için kullanılır ve modelin belirli özelliklerine ve projenin gereksinimlerine bağlı olarak tercih edilebilir. Jaccard İndeksi, özellikle sınıflandırma problemlerinde sınıf dengesizliği durumlarında etkilidir. Log Kaybı, özellikle olasılık tahminlerini değerlendirmek için kullanılırken, F1 Skoru hassasiyet ve duyarlılık arasındaki dengeyi gösterir.





